In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import re
from nltk.corpus import stopwords
import keras.preprocessing.text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import keras
# from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, SpatialDropout1D, Embedding, LSTM
from keras.callbacks import ModelCheckpoint,EarlyStopping

import xgboost as xgb

In [2]:
train = pd.read_csv("/kaggle/input/uw-cs480-fall20/train.csv")
test = pd.read_csv("/kaggle/input/uw-cs480-fall20/test.csv")

data = pd.concat([train, test])

data["gender_"] = pd.factorize(data["gender"])[0].astype(np.uint16)/4
data["baseColour_"] = pd.factorize(data["baseColour"])[0].astype(np.uint16)/45
data["season_"] = pd.factorize(data["season"])[0].astype(np.uint16)/3
data["usage_"] = pd.factorize(data["usage"])[0].astype(np.uint16)/6

data.head()

train_categorical_df = pd.DataFrame(data[:len(train)], columns = ['gender_','baseColour_','season_','usage_'])
test_categorical_df = pd.DataFrame(data[len(train):], columns = ['gender_','baseColour_','season_','usage_'])

In [7]:
category_list = train["category"].unique()
category_list.sort()
category_mapping = {label:idx for idx,label in enumerate(category_list)}
num_mapping = {idx:label for idx,label in enumerate(category_list)}

train_total_Y = train['category'][:len(train)].map(category_mapping)

train_Y = train['category'][:20000].map(category_mapping)
eval_Y = train['category'][20000:len(train)].map(category_mapping)

In [58]:
# xg_train = xgb.DMatrix(xgb_train_X, label=train_Y)
# xg_eval = xgb.DMatrix(xgb_eval_X, label=eval_Y)  

xg_train = xgb.DMatrix(train_categorical_df[:20000], label=train_Y)
xg_eval = xgb.DMatrix(train_categorical_df[20000:], label=eval_Y) 

train_categorical_df


# setup parameters for xgboost  
param = {}  
# use softmax multi-class classification  
param['objective'] = 'multi:softmax'  
# scale weight of positive examples  
param['eta'] = 0.1  
param['max_depth'] = 3  
# param['silent'] = 1  
param['nthread'] = 4  
param['num_class'] = 27  
  
# watchlist = [(xg_train,'train'), (xg_eval, 'eval') ]  
# num_round = 200  
# bst = xgb.train(param, xg_train, num_round, watchlist );

# # get prediction  
# pred = bst.predict(xg_eval); 

# print ('predicting, classification error=%f' % (1- sum(eval_Y == pred)/len(pred)))

In [81]:
xgb_classifier = xgb.XGBClassifier(nthread=4, max_depth=3, silent=0, 
                                   objective='multi:softmax', n_estimators=200)
xgb_classifier = xgb_classifier.fit(train_categorical_df[:20000], train_Y)

predictions = xgb_classifier.predict(train_categorical_df[20000:])
acc = sum(predictions == eval_Y) /len(eval_Y)
print(acc)

0.544560540872772


In [82]:
eval_pred = xgb_classifier.predict_proba(train_categorical_df[20000:])
eval_pred.shape

(1627, 27)

In [8]:
val_size = int(len(train)/5)

X_train1 = train_categorical_df[val_size:]
X_val1 = train_categorical_df[:val_size]

X_train2 = pd.concat([train_categorical_df.iloc[:val_size,], train_categorical_df.iloc[val_size*2:]])
X_val2 = train_categorical_df[val_size:val_size*2]

X_train3 = pd.concat([train_categorical_df.iloc[:val_size*2,], train_categorical_df.iloc[val_size*3:]])
X_val3 = train_categorical_df[val_size*2:val_size*3]

X_train4 = pd.concat([train_categorical_df.iloc[:val_size*3,], train_categorical_df.iloc[val_size*4:]])
X_val4 = train_categorical_df[val_size*3:val_size*4]

X_train5 = train_categorical_df.iloc[:val_size*4,]
X_val5 = train_categorical_df[val_size*4:]

Y_train1 = train_total_Y[val_size:]
Y_val1 = train_total_Y[:val_size]

Y_train2 = pd.concat([train_total_Y.iloc[:val_size,], train_total_Y.iloc[val_size*2:]])
Y_val2 = train_total_Y[val_size:val_size*2]

Y_train3 = pd.concat([train_total_Y.iloc[:val_size*2,], train_total_Y.iloc[val_size*3:]])
Y_val3 = train_total_Y[val_size*2:val_size*3]

Y_train4 = pd.concat([train_total_Y.iloc[:val_size*3,], train_total_Y.iloc[val_size*4:]])
Y_val4 = train_total_Y[val_size*3:val_size*4]

Y_train5 = train_total_Y.iloc[:val_size*4,]
Y_val5 = train_total_Y[val_size*4:]

print(Y_train5.shape,Y_val5.shape)
# print(X_test.shape,Y_test.shape)

(17300,) (4327,)


In [9]:
def xgb_train_second_stage(X_train, Y_train, X_val, Y_val):

    xgb_classifier = xgb.XGBClassifier(nthread=4, max_depth=3, silent=0, 
                                       objective='multi:softmax', n_estimators=200)
    xgb_classifier = xgb_classifier.fit(X_train, Y_train)

    predictions = xgb_classifier.predict(X_val)
    acc = sum(predictions == Y_val) /len(Y_val)
    print(acc)
    eval_pred = xgb_classifier.predict_proba(X_val)
    return xgb_classifier, eval_pred

In [11]:
xgb_classifier1, eval_pred_second1 = xgb_train_second_stage(X_train1, Y_train1, X_val1, Y_val1)
xgb_classifier2, eval_pred_second2 = xgb_train_second_stage(X_train2, Y_train2, X_val2, Y_val2)
xgb_classifier3, eval_pred_second3 = xgb_train_second_stage(X_train3, Y_train3, X_val3, Y_val3)
xgb_classifier4, eval_pred_second4 = xgb_train_second_stage(X_train4, Y_train4, X_val4, Y_val4)
xgb_classifier5, eval_pred_second5 = xgb_train_second_stage(X_train5, Y_train5, X_val5, Y_val5)

0.5320231213872832
0.5387283236994219
0.5452023121387283
0.5479768786127167
0.5474924890224174


In [12]:
train_xgb_first_pred = np.concatenate([eval_pred_second1, eval_pred_second2, eval_pred_second3, eval_pred_second4,eval_pred_second5])

In [10]:
xgb_classifier_second_stage, eval_pred = xgb_train_second_stage(train_categorical_df, train_total_Y, X_val1, Y_val1)
test_xgb_first_pred = xgb_classifier_second_stage.predict_proba(test_categorical_df)

0.5359537572254335


In [12]:
pd.DataFrame(train_xgb_first_pred).to_csv("category_xgb_train.csv", index=False)
pd.DataFrame(test_xgb_first_pred).to_csv("category_xgb_test.csv", index=False)